Libraries to be imported:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import requests # library to handle requests

! conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    beautifulsoup4-4.8.1       |           py36_0         153 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  a

Scrapping the wikipedia webpage.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)

soup = BeautifulSoup(r.content, 'html5lib')

table = soup.find('div', attrs = {'id':'container'})

print('Page scrapped.')

Page scrapped.


Extracting table contents from wikipedia page.

In [3]:
postalCodes = []
boroughs = []
neighborhoods = []
colNum = 1
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string)>2:
            passVal = False
            if colNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string)
                    colNum = 2
                else:
                    continue
            elif colNum == 2:
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    colNum = 1
                    continue
                else:
                    boroughs.append(cell.string)
                    colNum = 3
            elif colNum == 3:
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string)
                colNum = 1
                
print('Data Collected.')

Data Collected.


Define column names and create empty dataframe.

In [4]:
col_names = ['PostalCode','Borough','Neighborhood']

tordf = pd.DataFrame(columns = col_names)

tordf

,PostalCode,Borough,Neighborhood


Add extracted data to the dataframe columns.

In [5]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    tordf = tordf.append({'PostalCode':code, 'Borough':borough, 'Neighborhood':neighborhood_name},
                                ignore_index = True)

In [6]:
tordf.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North\n


Note that there is no 'Regent Park' neighborhood for M5A postal code in the wikipedia page table.

In [7]:
tordf = tordf.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(lambda x:', '.join(x))

In [8]:
tordf.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [9]:
tordf.shape

(103, 3)